In [66]:
import osmnx
import math
import heapq
from search_optimization_tools import Node
osmnx.utils.config(requests_kwargs={'verify':False})
reference = (43.661667, -79.395)
G = osmnx.graph_from_point(reference, dist=300, clean_periphery=True, simplify=True)

In [67]:
len(G.edges)+len(G.nodes)

628

In [68]:
def Dijkstra(G, origin, destination):
    # Using a set here avoids the problem with self loops
    seen = set()
    shortest_dist = {osmid: math.inf for osmid in G.nodes()}
    unrelaxed_nodes = [Node(graph=G, osmid=osmid) for osmid in G.nodes()]
    shortest_dist[origin.osmid] = 0
    found = False
    route = None
    while len(unrelaxed_nodes) > 0 and not found:
        node = min(unrelaxed_nodes, key=lambda node: shortest_dist[node.osmid])
        # relaxing the node, so this node's value in shortest_dist is the shortest distance between the origin and destination
        unrelaxed_nodes.remove(node)
        seen.add(node.osmid)
        # if the destination node has been relaxed then that is the route we want
        if node == destination:
            route = node.path()
            found = True
            continue
        # otherwise, let's relax edges of its neighbours
        for child in node.expand():
            # skip self-loops
            if child.osmid in seen:
                continue
            child_obj = next(
                (node for node in unrelaxed_nodes if node.osmid == child.osmid), None
            )
            child_obj.distance = child.distance
            distance = shortest_dist[node.osmid] + child.distance
            if distance < shortest_dist[child_obj.osmid]:
                shortest_dist[child_obj.osmid] = distance
                child_obj.parent = node
    return route

In [69]:
def UCS(origin, destination):
    entry_count = 1
    priority_queue = [(0, 0, origin)]
    found = False
    visited = []
    route = None
    while priority_queue and not found:
        node = heapq.heappop(priority_queue)
        node_cost = node[0]
        node = node[2]
        if node in visited:
            continue
        visited.append(node)
        # We found the destination
        if node == destination:
            route = node.path()
            found = True
            continue
        for child in node.expand():
            total_cost = child.distance + node_cost
            matches = [item for item in priority_queue if item[2] == child]
            if matches:
                # Update the entry if the new priority is better
                if total_cost < matches[0][0]:
                    priority_queue[priority_queue.index(matches[0])] = (
                        total_cost,
                        entry_count,
                        child,
                    )
                    entry_count += 1
                    heapq.heapify(priority_queue)
            else:
                heapq.heappush(priority_queue, (total_cost, entry_count, child))
                entry_count += 1
    return route

In [70]:
from time import process_time

s = process_time()
for node1 in G.nodes:
    for node2 in G.nodes:
        Dijkstra(G,Node(G,node1),Node(G,node2))
e = process_time()

e-s


84.609375

In [71]:
s = process_time()
for node1 in G.nodes:
    for node2 in G.nodes:
        UCS(Node(G,node1),Node(G,node2))
e = process_time()

e-s

58.234375